In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from time import sleep

for i in range(100):
    print(i)
    sleep(1)  # Pause for 1 second between each iteration


In [3]:
import pandas as pd

df=pd.read_csv('/content/JobsDataset.csv')


print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           10000 non-null  int64 
 1   Query        10000 non-null  object
 2   Job Title    10000 non-null  object
 3   Description  10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB
None


,ID,Query,Job Title,Description
0,1,Data Scientist,Junior Data Scientist Apprenticeship,Job Description As a Junior Data Scientist at ...
1,2,Data Scientist,"HBO Data Scientist, Content Science",OVERALL SUMMARY As a Data Scientist on the Dat...
2,3,Data Scientist,Junior Data Scientist,The Team: The Data science team is a newly for...
3,4,Data Scientist,Jr Data Scientist,We now have a need for junior Data Scientist(s...
4,5,Data Scientist,"Data Scientist, Premium Content",Do you want to help guide the core business of...


In [4]:
print(df['Query'].value_counts())

Query
Cloud Architect                  540
Network Architect                511
Full Stack Developer             400
Database Administrator           400
Data Scientist                   400
Technology Integration           399
Data Architect                   399
Data and Analytics Manager       399
IT Systems Administrator         399
IT Consultant                    398
Data Analyst                     397
Business Analyst                 396
Cloud Services Developer         395
Information Security Analyst     395
Data Quality Manager             394
Technical Operations             394
Machine Learning                 392
Statistics                       390
Data Engineer                    385
Data Warehousing                 385
Artificial Intelligence          382
Deep Learning                    381
Data Visualization Expert        377
Business Intelligence Analyst    372
Big Data Engineer                320
Name: count, dtype: int64


In [5]:
df=df[['Query','Description']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Query        10000 non-null  object
 1   Description  10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [6]:
import re
df['Description'] = df['Description'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))
df['Description'] = df['Description'].apply((lambda x: x.lower()))

In [7]:
df.head()

,Query,Description
0,Data Scientist,job description as a junior data scientist at ...
1,Data Scientist,overall summary as a data scientist on the dat...
2,Data Scientist,the team the data science team is a newly form...
3,Data Scientist,we now have a need for junior data scientists ...
4,Data Scientist,do you want to help guide the core business of...


In [8]:
df['Query'].value_counts()

Query
Cloud Architect                  540
Network Architect                511
Full Stack Developer             400
Database Administrator           400
Data Scientist                   400
Technology Integration           399
Data Architect                   399
Data and Analytics Manager       399
IT Systems Administrator         399
IT Consultant                    398
Data Analyst                     397
Business Analyst                 396
Cloud Services Developer         395
Information Security Analyst     395
Data Quality Manager             394
Technical Operations             394
Machine Learning                 392
Statistics                       390
Data Engineer                    385
Data Warehousing                 385
Artificial Intelligence          382
Deep Learning                    381
Data Visualization Expert        377
Business Intelligence Analyst    372
Big Data Engineer                320
Name: count, dtype: int64

In [9]:
df_balanced=pd.DataFrame()
unique_queries=df['Query'].unique()
for i in unique_queries:
  subset=df[df['Query']==i].sample(500,replace=True)
  df_balanced=pd.concat([df_balanced,subset])
df_balanced.reset_index(drop=True,inplace=True)
print(df_balanced['Query'].value_counts())
print(df_balanced.info())

Query
Data Scientist                   500
Data Analyst                     500
Data Architect                   500
Data Engineer                    500
Statistics                       500
Database Administrator           500
Business Analyst                 500
Data and Analytics Manager       500
Machine Learning                 500
Artificial Intelligence          500
Deep Learning                    500
Business Intelligence Analyst    500
Data Visualization Expert        500
Data Quality Manager             500
Big Data Engineer                500
Data Warehousing                 500
Technology Integration           500
IT Consultant                    500
IT Systems Administrator         500
Cloud Architect                  500
Technical Operations             500
Cloud Services Developer         500
Full Stack Developer             500
Information Security Analyst     500
Network Architect                500
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Range

In [10]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

df_balanced['Query']=le.fit_transform(df_balanced['Query'])

In [11]:
le.classes_

array(['Artificial Intelligence', 'Big Data Engineer', 'Business Analyst',
       'Business Intelligence Analyst', 'Cloud Architect',
       'Cloud Services Developer', 'Data Analyst', 'Data Architect',
       'Data Engineer', 'Data Quality Manager', 'Data Scientist',
       'Data Visualization Expert', 'Data Warehousing',
       'Data and Analytics Manager', 'Database Administrator',
       'Deep Learning', 'Full Stack Developer', 'IT Consultant',
       'IT Systems Administrator', 'Information Security Analyst',
       'Machine Learning', 'Network Architect', 'Statistics',
       'Technical Operations', 'Technology Integration'], dtype=object)

In [12]:
import random
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')


def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(synonyms)
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    sentence = ' '.join(new_words)
    return sentence

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.add(l.name())
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

def random_insertion(sentence, n):
    words = sentence.split()
    for _ in range(n):
        new_word = get_synonyms(random.choice(words))
        if new_word:
            words.insert(random.randint(0, len(words)), random.choice(new_word))
    return ' '.join(words)

def random_swap(sentence, n):
    words = sentence.split()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def random_deletion(sentence, p):
    words = sentence.split()
    if len(words) == 1:  # return if single word
        return sentence

    new_words = []
    for word in words:
        if random.uniform(0, 1) > p:
            new_words.append(word)
    if len(new_words) == 0:  # ensure at least one word remains
        new_words.append(random.choice(words))
    return ' '.join(new_words)

# Example usage:
sentence = "This is a sample sentence for data augmentation."

sentence = "john has not progressed in his position he is continuously late leaves early and takes many breaks throughout the day he calls out at least every other week and its always on fridays his performance has significantly declined my suggestion is he is not suitable for this position"

print("Original Sentence:", sentence)
print("Synonym Replacement:", synonym_replacement(sentence, 2))
print("Random Insertion:", random_insertion(sentence, 2))
print("Random Swap:", random_swap(sentence, 2))
print("Random Deletion:", random_deletion(sentence, 0.3))




[nltk_data] Downloading package wordnet to /root/nltk_data...


Original Sentence: john has not progressed in his position he is continuously late leaves early and takes many breaks throughout the day he calls out at least every other week and its always on fridays his performance has significantly declined my suggestion is he is not suitable for this position
Synonym Replacement: privy has not progressed in his position he is continuously late leaves early and takes many breaks throughout the day he calls out at least every other week and its always on fridays his performance has importantly declined my suggestion is he is not suitable for this position
Random Insertion: john has not progressed in his position he is continuously late leaves early and takes many breaks throughout the day he calls out at least every other week and its always on fridays his performance has significantly declined my suggestion is he is not suitable for this position
Random Swap: john has not progressed in his position he is continuously late leaves early and takes man

In [13]:
import numpy as np
# Original data
original_sentences = list(df_balanced['Description'].values)
original_labels = list(df_balanced['Query'].values)

# Augmented data
augmented_sentences = []
for sentence in original_sentences:
    augmented_sentences.append(synonym_replacement(sentence, 2))
    augmented_sentences.append(random_insertion(sentence, 2))
    augmented_sentences.append(random_swap(sentence, 2))
    augmented_sentences.append(random_deletion(sentence, 0.3))

# Combine original and augmented data
all_sentences = original_sentences + augmented_sentences

# Repeat labels for the augmented sentences
augmented_labels = np.repeat(original_labels, 4)  # assuming each sentence generates 4 augmented versions
all_labels = np.concatenate((original_labels, augmented_labels))

print(len(all_sentences),len(all_labels))

62500 62500


In [14]:
# m=0
# for i in all_sentences:
#   m=max(m,len(i.split()))
# print("Maximum Words",m)

In [15]:
m=0
for i in all_sentences:
  m=max(m,len(i.split()))
print("Maximum Words",m)

Maximum Words 2574


In [16]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(all_sentences,all_labels,test_size=0.1,random_state=50)

In [17]:
print("Training Samples",len(x_train))
print("Testing Samples",len(x_test))

Training Samples 56250
Testing Samples 6250


In [18]:
x_train[:5],y_train[:5]

(['overview draper is an independent nonprofit research and development company headquartered in cambridge ma the 1700 employees of draper tackle important national challenges with a promise of delivering successful and usable solutions from military defense and space exploration to biomedical engineering lives often depend on the solutions we provide our multidisciplinary teams of engineers and scientists work in a collaborative environment that inspires the crossfertilization of ideas necessary for true innovation for more information about draper visit wwwdrapercom our work is very important to us but so is our life outside of work draper supports many programs to improve worklife balance including workplace flexibility employee clubs ranging from photography to yoga health and finance workshops off site social events and discounts to local museums and cultural activities if this specific job opportunity and the chance to work at a nationally renowned rd innovation company appeals t

In [2]:
# !pip install sentence-transformers datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [21]:
from datasets import Dataset
train_df = pd.DataFrame({'text': x_train, 'label': y_train})
train_dataset = Dataset.from_pandas(train_df)
test_df = pd.DataFrame({'text': x_test, 'label': y_test})
test_dataset = Dataset.from_pandas(test_df)

In [22]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/56250 [00:00<?, ? examples/s]

Map:   0%|          | 0/6250 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [24]:
!pip install evaluate

import evaluate

accuracy = evaluate.load("accuracy")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [25]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [26]:
# 'Artificial Intelligence', 'Big Data Engineer', 'Business Analyst',
#        'Business Intelligence Analyst', 'Cloud Architect',
#        'Cloud Services Developer', 'Data Analyst', 'Data Architect',
#        'Data Engineer', 'Data Quality Manager', 'Data Scientist',
#        'Data Visualization Expert', 'Data Warehousing',
#        'Data and Analytics Manager', 'Database Administrator',
#        'Deep Learning', 'Full Stack Developer', 'IT Consultant',
#        'IT Systems Administrator', 'Information Security Analyst',
#        'Machine Learning', 'Network Architect', 'Statistics',
#        'Technical Operations', 'Technology Integration'

In [27]:
# 'Artificial Intelligence', 'Big Data Engineer', 'Business Analyst',
#        'Business Intelligence Analyst', 'Data Analyst', 'Data Architect',
#        'Data Engineer', 'Data Quality Manager', 'Data Scientist',
#        'Data Visualization Expert', 'Data and Analytics Manager',
#        'Database Administrator', 'Deep Learning', 'Machine Learning',
#        'Statistics'

In [28]:
# id2label = {
#     0:'Artificial Intelligence',
#     1:'Business Analyst',
#     2:'Data Analyst',
#     3:'Data Architect',
#     4:'Data Engineer',
#     5:'Data Scientist',
#     6:'Data and Analytics Manager',
#     7:'Database Administrator',
#     8:'Machine Learning',
#     9:'Statistics'
#             }
# label2id = {
#     'Artificial Intelligence':0,
#     'Business Analyst':1,
#     'Data Analyst':2,
#     'Data Architect':3,
#     'Data Engineer':4,
#     'Data Scientist':5,
#     'Data and Analytics Manager':6,
#     'Database Administrator':7,
#     'Machine Learning':8,
#     'Statistics':9
# }

In [29]:
id2label = {
    0:'Artificial Intelligence',
    1:'Big Data Engineer',
    2:'Business Analyst',
    3:'Business Intelligence Analyst',
    4:'Cloud Architect',
    5:'Cloud Services Developer',
    6:'Data Analyst',
    7:'Data Architect',
    8:'Data Engineer',
    9:'Data Quality Manager',
    10:'Data Scientist',
    11:'Data Visualization Expert',
    12:'Data Warehousing',
    13:'Data and Analytics Manager',
    14:'Database Administrator',
    15:'Deep Learning',
    16:'Full Stack Developer',
    17:'IT Consultant',
    18:'IT Systems Administrator',
    19:'Information Security Analyst',
    20:'Machine Learning',
    21:'Network Architect',
    22:'Statistics',
    23:'Technical Operations',
    24:'Technology Integration'
            }
label2id = {
    'Artificial Intelligence':0,
    'Big Data Engineer':1,
    'Business Analyst':2,
    'Business Intelligence Analyst':3,
    'Cloud Architect':4,
    'Cloud Services Developer':5,
    'Data Analyst':6,
    'Data Architect':7,
    'Data Engineer':8,
    'Data Quality Manager':9,
    'Data Scientist':10,
    'Data Visualization Expert':11,
    'Data Warehousing':12,
    'Data and Analytics Manager':13,
    'Database Administrator':14,
    'Deep Learning':15,
    'Full Stack Developer':16,
    'IT Consultant':17,
    'IT Systems Administrator':18,
    'Information Security Analyst':19,
    'Machine Learning':20,
    'Network Architect':21,
    'Statistics':22,
    'Technical Operations':23,
    'Technology Integration':24
}

In [30]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 3
batches_per_epoch = len(tokenized_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [31]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=25, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [32]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_test,
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [33]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [34]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [35]:
callbacks = [metric_callback]

In [36]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3515/3515 [==============================] - 3562s 998ms/step - loss: 1.0198 - val_loss: 0.4205 - accuracy: 0.8778
Epoch 2/3
3515/3515 [==============================] - 3527s 1s/step - loss: 0.2844 - val_loss: 0.1989 - accuracy: 0.9470
Epoch 3/3
3515/3515 [==============================] - 3489s 993ms/step - loss: 0.1318 - val_loss: 0.1485 - accuracy: 0.9586


In [39]:
data = {
    'Query': all_sentences,
    'Description': all_labels
}
df = pd.DataFrame(data)
df.to_csv('/content/drive/MyDrive/AugmentedJobsDataset.csv')

In [38]:
model.save_pretrained('/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples')
tokenizer.save_pretrained('/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples')

('/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples/tokenizer_config.json',
 '/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples/special_tokens_map.json',
 '/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples/vocab.txt',
 '/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples/added_tokens.json',
 '/content/drive/MyDrive/query_job_description_model_25_95.86acc_0.1485val_loss_500samples/tokenizer.json')

In [ ]:
text=input("Enter Feedback:")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/query_job_description_model")
inputs = tokenizer(text, return_tensors="tf")

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/query_job_description_model")
logits = model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]